In [28]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import itertools

In [29]:
stockin=input('Enter Stock to be evaluated')
ticker=yf.Ticker(stockin)
today = datetime.now()
prev_day = today - timedelta(days=1)
prev_day_str = prev_day.strftime('%Y-%m-%d')

hist = ticker.history(start=prev_day_str, end=today.strftime('%Y-%m-%d'), interval='1d')

if not hist.empty:
    prev_open = hist['Open'].iloc[0]
    prev_close = hist['Close'].iloc[0]
    print(f"Previous day open: {prev_open}")
    print(f"Previous day close: {prev_close}")
else:
    print("No data for previous day.")

Previous day open: 475.0199890136719
Previous day close: 478.8699951171875


In [30]:
prev_day_mean=(prev_close+prev_open)/2
prev_day_mean

476.9449920654297

In [31]:
df = yf.download(tickers=stockin, period='1d', interval='1m')

df.dropna(inplace=True)

df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()

current_price = df['Close'].iloc[-1]
rolling_mean_20min = df['RollingMean_20'].iloc[-1]
rolling_std_20min = df['RollingStd_20'].iloc[-1]

print("Current Price:          ", current_price)
print("20-min Rolling Mean:    ", rolling_mean_20min)
print("20-min Rolling Std Dev: ", rolling_std_20min)

C:\Users\sauri\AppData\Local\Temp\ipykernel_21780\1151132504.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=stockin, period='1d', interval='1m')
[*********************100%***********************]  1 of 1 completed

Current Price:           Ticker
MSFT    479.029999
Name: 2025-06-12 19:59:00+00:00, dtype: float64
20-min Rolling Mean:     478.7057510375977
20-min Rolling Std Dev:  0.40406033913219547


In [32]:
z_score=(current_price-rolling_mean_20min) / rolling_std_20min
max_vol_20min = df['Volume'].shift(1).tail(20).max()
current_volume = df['Volume'].iloc[-1]

print("Current Tick Volume:", current_volume)
print("Max Volume in Last 20 Min (excluding current tick):", max_vol_20min)
print("z-score",z_score)

Current Tick Volume: Ticker
MSFT    393247
Name: 2025-06-12 19:59:00+00:00, dtype: int64
Max Volume in Last 20 Min (excluding current tick): Ticker
MSFT    213952.0
dtype: float64
z-score Ticker
MSFT    0.802474
Name: 2025-06-12 19:59:00+00:00, dtype: float64


In [33]:
# Initial capital and trading parameters
amount = 10000
initial_amount = amount
position = 0
shares_held = 0
buy_percentage = 0.95
sell_percentage = 0.35
target_profit = initial_amount * 0.20 

In [34]:
def buy_sig(z_score, max_vol_20min, current_volume):
    z_score = float(z_score.iloc[-1]) if hasattr(z_score, 'iloc') else float(z_score)
    max_vol_20min = float(max_vol_20min.iloc[-1]) if hasattr(max_vol_20min, 'iloc') else float(max_vol_20min)
    current_volume = float(current_volume.iloc[-1]) if hasattr(current_volume, 'iloc') else float(current_volume)
    
    signal = 0
    signal_type = "HOLD"
    
    volume_margin = abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15
    
    if z_score <= -0.3:
        signal = 1
        signal_type = "BUY"
    elif z_score > 1.3:
        signal = -1  
        signal_type = "SELL"

    elif volume_margin and (z_score <= 1.3) and (z_score>-0.3):
            signal = -1
            signal_type = "SELL"
    elif volume_margin and (z_score>-0.3):
         signal=1
         signal_type="BUY"
    
    print(f"\nSignal Analysis:")
    print(f"Z-Score: {z_score:.4f}")
    print(f"Signal: {signal} ({signal_type})")
    print(f"Current Volume: {current_volume}")
    print(f"Max Volume (20min): {max_vol_20min}")
    print(f"Volume within 20% margin: {volume_margin}")
    
    return signal

In [35]:
# Get trading signal
signal = buy_sig(z_score, max_vol_20min, current_volume)

# Convert Series to float if needed
current_price = float(current_price.iloc[-1]) if hasattr(current_price, 'iloc') else float(current_price)

# Execute trades based on signal
if signal == 1 and position == 0:  # Buy signal
    investment = amount * buy_percentage
    shares_to_buy = int(investment // current_price)  # Convert to int
    if shares_to_buy > 0:
        position = current_price * shares_to_buy
        shares_held = shares_to_buy
        amount -= position
        print(f"Buying {shares_to_buy} shares at {current_price}")
        print(f"Position: {position}, Remaining cash: {amount}")

elif signal == -1 and position > 0:  # Sell signal
    # If z-score > 1.8 and volume condition met, sell all shares
    if z_score > 1.8 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.20:
        shares_to_sell = shares_held  # Sell all shares
    else:
        shares_to_sell = int(shares_held * sell_percentage)  # Sell normal percentage
        
    if shares_to_sell > 0:
        sale_value = current_price * shares_to_sell
        amount += sale_value
        shares_held -= shares_to_sell
        position = current_price * shares_held
        print(f"Selling {shares_to_sell} shares at {current_price}")
        print(f"Position: {position}, Available cash: {amount}")

# Check termination conditions
total_value = amount + position
profit = total_value - initial_amount

if amount <= 0:
    print("Trading stopped: No funds available")
elif profit >= target_profit:
    print(f"Trading stopped: Profit target reached. Total profit: {profit}")

print(f"Current total value: {total_value}")
print(f"Current profit/loss: {profit}")


Signal Analysis:
Z-Score: 0.8025
Signal: 0 (HOLD)
Current Volume: 393247.0
Max Volume (20min): 213952.0
Volume within 20% margin: False
Current total value: 10000
Current profit/loss: 0


### MAIN CODE RUN

In [ ]:
def is_market_open():
    # Indian market hours (9:15 AM to 3:30 PM IST)
    now = datetime.now()
    market_start = now.replace(hour=9, minute=15, second=0)
    market_end = now.replace(hour=15, minute=30, second=0)
    return market_start <= now <= market_end and now.weekday() < 5

def wait_for_next_minute():
    """Wait until the start of the next minute"""
    now = datetime.now()
    # Wait until the next minute
    wait_seconds = 60 - now.second
    if wait_seconds > 0:
        print(f"Synchronizing with market ticks... waiting {wait_seconds} seconds")
        time.sleep(wait_seconds)

def run_trading_strategy():
    global amount, position, shares_held
    trades = []
    
    # Synchronize with market ticks
    wait_for_next_minute()
    
    while True:
        try:
            if not is_market_open():
                print("Market is closed. Waiting...")
                time.sleep(60)
                continue
            
            end_time = datetime.now().replace(second=0, microsecond=0)
            start_time = end_time - timedelta(hours=1)
            
            start_timestamp = int(start_time.timestamp())
            end_timestamp = int(end_time.timestamp())
            
            try:
                df = yf.download(
                    tickers=stockin,
                    start=start_timestamp,
                    end=end_timestamp,
                    interval='1m',
                    progress=False
                )
            except Exception as e:
                print(f"Download error: {e}")
                time.sleep(60)
                continue
            
            if df.empty:
                print("No data received, waiting for next tick...")
                time.sleep(60)
                continue

            df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
            df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
            
            current_price = float(df['Close'].iloc[-1])
            rolling_mean_20min = df['RollingMean_20'].iloc[-1]
            rolling_std_20min = df['RollingStd_20'].iloc[-1]
            z_score = (current_price - rolling_mean_20min) / rolling_std_20min
            
            recent_volumes = df['Volume'].tail(20)
            non_zero_volumes = recent_volumes[recent_volumes > 0]
            max_vol_20min = non_zero_volumes.max() if not non_zero_volumes.empty else 0
            current_volume = float(df['Volume'].iloc[-1])
            
            if current_volume == 0:
                print("Warning: Current volume is 0, using last non-zero volume")
                current_volume = non_zero_volumes.iloc[-1] if not non_zero_volumes.empty else 0
            
            signal = buy_sig(z_score, max_vol_20min, current_volume)
            
            if signal == 1 and position == 0:  
                investment = amount * buy_percentage
                shares_to_buy = int(investment // current_price)
                if shares_to_buy > 0:
                    position = current_price * shares_to_buy
                    shares_held = shares_to_buy
                    amount -= position
                    trades.append({
                        'time': datetime.now(),
                        'type': 'BUY',
                        'shares': shares_to_buy,
                        'price': current_price,
                        'value': position,
                        'remaining_cash': amount
                    })
                    print(f"\nTRADE EXECUTED - BUY:")
                    print(f"Shares: {shares_to_buy}")
                    print(f"Price: {current_price:.2f}")
                    print(f"Position: {position:.2f}")
                    print(f"Remaining Cash: {amount:.2f}")

            elif signal == -1 and position > 0:  
                shares_to_sell = int(shares_held * sell_percentage)
                if shares_to_sell > 0:
                    sale_value = current_price * shares_to_sell
                    amount += sale_value
                    shares_held -= shares_to_sell
                    position = current_price * shares_held
                    trades.append({
                        'time': datetime.now(),
                        'type': 'SELL',
                        'shares': shares_to_sell,
                        'price': current_price,
                        'value': sale_value,
                        'remaining_cash': amount
                    })
                    print(f"\nTRADE EXECUTED - SELL:")
                    print(f"Shares: {shares_to_sell}")
                    print(f"Price: {current_price:.2f}")
                    print(f"Position: {position:.2f}")
                    print(f"Available Cash: {amount:.2f}")

            total_value = amount + position
            profit = total_value - initial_amount
            
            print(f"\nPORTFOLIO STATUS:")
            print(f"Time: {datetime.now()}")
            print(f"Current Price: {current_price:.2f}")
            print(f"Shares Held: {shares_held}")
            print(f"Position Value: {position:.2f}")
            print(f"Cash: {amount:.2f}")
            print(f"Total Value: {total_value:.2f}")
            print(f"Profit/Loss: {profit:.2f}")
            print("-" * 50)
            
            if amount <= 0:
                print("\nTrading stopped: No funds available")
                break
            elif profit >= target_profit:
                print(f"\nTrading stopped: Profit target reached. Total profit: {profit:.2f}")
                break
            
            if trades:
                print("\nTRADE HISTORY:")
                for trade in trades[-3:]:  # Show last 3 trades
                    print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ {trade['price']:.2f}")
            
            wait_for_next_minute()
            
        except Exception as e:
            print(f"Error occurred: {e}")
            time.sleep(60)

run_trading_strategy()

### BACKTESTING


In [36]:
def backtest_strategy(symbol, start_date, end_date, initial_capital=10000):
    # Download historical data
    df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
    
    if df.empty:
        print('No data available for the specified period')
        return
    
    # Initialize variables
    amount = initial_capital
    initial_amount = amount
    position = 0
    shares_held = 0
    trades = []
    
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    
    df = df.dropna()
    
    for i in range(len(df)):
        current_price = float(df['Close'].iloc[i])
        rolling_mean_20min = df['RollingMean_20'].iloc[i]
        rolling_std_20min = df['RollingStd_20'].iloc[i]
        
        z_score = (current_price - rolling_mean_20min) / rolling_std_20min
        
        if i >= 20:
            recent_volumes = df['Volume'].iloc[i-20:i]
            max_vol_20min = recent_volumes.max()
        else:
            max_vol_20min = df['Volume'].iloc[:i].max()
        
        current_volume = df['Volume'].iloc[i]
        
        signal = buy_sig(z_score, max_vol_20min, current_volume)
        
        if signal == 1 and position == 0:  
            investment = amount * buy_percentage
            shares_to_buy = int(investment // current_price)
            if shares_to_buy > 0:
                position = current_price * shares_to_buy
                shares_held = shares_to_buy
                amount -= position
                trades.append({
                    'time': df.index[i],
                    'type': 'BUY',
                    'shares': shares_to_buy,
                    'price': current_price,
                    'value': position,
                    'remaining_cash': amount
                })

        elif signal == -1 and position > 0:  
            shares_to_sell = int(shares_held * sell_percentage)
            if shares_to_sell > 0:
                sale_value = current_price * shares_to_sell
                amount += sale_value
                shares_held -= shares_to_sell
                position = current_price * shares_held
                trades.append({
                    'time': df.index[i],
                    'type': 'SELL',
                    'shares': shares_to_sell,
                    'price': current_price,
                    'value': sale_value,
                    'remaining_cash': amount
                })
        
        if shares_held > 0:
            position = current_price * shares_held
    
    total_value = amount + position
    total_return = ((total_value - initial_capital) / initial_capital) * 100
    
    print(f'\nBacktest Results ({start_date} to {end_date}):')
    print(f'Initial Capital: ${initial_amount:,.2f}')
    print(f'Final Capital: ${total_value:,.2f}')
    print(f'Total Return: {total_return:.2f}%')
    print(f'Number of Trades: {len(trades)}')
    
    # Print trade history
    print('\nTrade History:')
    for trade in trades:
        print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ ${trade['price']:.2f}")
    
    return df, trades

In [38]:
from datetime import datetime, timedelta

end_date = datetime.now()
start_date = end_date - timedelta(days=3)  

df, trades = backtest_strategy(stockin, start_date, end_date, initial_capital=10000)

C:\Users\sauri\AppData\Local\Temp\ipykernel_21780\1373793095.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
[*********************100%***********************]  1 of 1 completed
C:\Users\sauri\AppData\Local\Temp\ipykernel_21780\1373793095.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])



Signal Analysis:
Z-Score: -1.8062
Signal: 1 (BUY)
Current Volume: 33996.0
Max Volume (20min): nan
Volume within 20% margin: False

Signal Analysis:
Z-Score: -1.1337
Signal: 1 (BUY)
Current Volume: 28529.0
Max Volume (20min): 33996.0
Volume within 20% margin: False

Signal Analysis:
Z-Score: -0.8144
Signal: 1 (BUY)
Current Volume: 29014.0
Max Volume (20min): 33996.0
Volume within 20% margin: True

Signal Analysis:
Z-Score: -0.6290
Signal: 1 (BUY)
Current Volume: 30577.0
Max Volume (20min): 33996.0
Volume within 20% margin: True

Signal Analysis:
Z-Score: -0.4661
Signal: 1 (BUY)
Current Volume: 36571.0
Max Volume (20min): 33996.0
Volume within 20% margin: True

Signal Analysis:
Z-Score: -0.2167
Signal: -1 (SELL)
Current Volume: 35964.0
Max Volume (20min): 36571.0
Volume within 20% margin: True

Signal Analysis:
Z-Score: 0.0135
Signal: 0 (HOLD)
Current Volume: 19559.0
Max Volume (20min): 36571.0
Volume within 20% margin: False

Signal Analysis:
Z-Score: -0.0743
Signal: 0 (HOLD)
Current V

### Z-Score Optimization

In [ ]:
def optimize_z_scores(symbol, start_date, end_date, initial_capital=10000):
    buy_thresholds = np.arange(-1.5, -0.2, 0.1)   
    sell_thresholds = np.arange(1.1, 2.0, 0.1)   
    
    best_return = -float('inf')
    best_params = None
    results = []
    
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
        if df.empty:
            print(f'No data available for {symbol} in the specified period')
            return None, pd.DataFrame()
    except Exception as e:
        print(f'Error downloading data: {e}')
        return None, pd.DataFrame()
    
    def modified_buy_sig(z_score, max_vol_20min, current_volume, buy_threshold, sell_threshold):
        z_score = float(z_score.iloc[-1]) if hasattr(z_score, 'iloc') else float(z_score)
        max_vol_20min = float(max_vol_20min.iloc[-1]) if hasattr(max_vol_20min, 'iloc') else float(max_vol_20min)
        current_volume = float(current_volume.iloc[-1]) if hasattr(current_volume, 'iloc') else float(current_volume)
        
        signal = 0
        
        if z_score <= buy_threshold:
            signal = 1
        elif z_score >= sell_threshold:
            signal = -1
        elif (z_score > 0) and (z_score <= sell_threshold):
            if max_vol_20min > 0 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15:
                signal = 1
        
        return signal
    
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    df = df.dropna()
    
    print(f'Testing {len(buy_thresholds) * len(sell_thresholds)} combinations...')
    
    for buy_thresh in buy_thresholds:
        for sell_thresh in sell_thresholds:
            amount = initial_capital
            position = 0
            shares_held = 0
            trades = []
            
            for i in range(len(df)):
                current_price = float(df['Close'].iloc[i])
                rolling_mean = df['RollingMean_20'].iloc[i]
                rolling_std = df['RollingStd_20'].iloc[i]
                z_score = (current_price - rolling_mean) / rolling_std
                
                if i >= 20:
                    recent_volumes = df['Volume'].iloc[i-20:i]
                    max_vol_20min = float(recent_volumes.max())
                else:
                    max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
                
                current_volume = float(df['Volume'].iloc[i])
                
                signal = modified_buy_sig(z_score, max_vol_20min, current_volume, buy_thresh, sell_thresh)
                
                if signal == 1 and position == 0:
                    investment = amount * buy_percentage
                    shares_to_buy = int(investment // current_price)
                    if shares_to_buy > 0:
                        position = current_price * shares_to_buy
                        shares_held = shares_to_buy
                        amount -= position
                        trades.append({
                            'time': df.index[i],
                            'type': 'BUY',
                            'shares': shares_to_buy,
                            'price': current_price
                        })
                
                elif signal == -1 and position > 0:
                    shares_to_sell = int(shares_held * sell_percentage)
                    if shares_to_sell > 0:
                        sale_value = current_price * shares_to_sell
                        amount += sale_value
                        shares_held -= shares_to_sell
                        position = current_price * shares_held
                        trades.append({
                            'time': df.index[i],
                            'type': 'SELL',
                            'shares': shares_to_sell,
                            'price': current_price
                        })
            
            final_position_value = shares_held * float(df['Close'].iloc[-1])
            total_value = amount + final_position_value
            total_return = ((total_value - initial_capital) / initial_capital) * 100
            
            results.append({
                'buy_threshold': buy_thresh,
                'sell_threshold': sell_thresh,
                'total_return': total_return,
                'n_trades': len(trades)
            })
            
            if total_return > best_return:
                best_return = total_return
                best_params = {
                    'buy_threshold': buy_thresh,
                    'sell_threshold': sell_thresh,
                    'total_return': total_return,
                    'n_trades': len(trades)
                }
    
    results_df = pd.DataFrame(results)
    return best_params, results_df

In [ ]:
end_date = datetime.now()
start_date = end_date - timedelta(days=5)  

print('Running optimization for the last 5 days...')
print('Note: Using 1-minute data from the last 5 trading days due to Yahoo Finance API limitations')
best_params, results_df = optimize_z_scores(stockin, start_date, end_date, initial_capital=10000)

print('\nBest Parameters Found:')
print(f"Buy Threshold (z-score): {best_params['buy_threshold']:.2f}")
print(f"Sell Threshold (z-score): {best_params['sell_threshold']:.2f}")
print(f"Total Return: {best_params['total_return']:.2f}%")
print(f"Number of Trades: {best_params['n_trades']}")


print('\nTop 5 Combinations:')
print(results_df.sort_values('total_return', ascending=False).head())

import seaborn as sns
import matplotlib.pyplot as plt

heatmap_data = results_df.pivot(index='buy_threshold', columns='sell_threshold', values='total_return')

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, cmap='RdYlGn', center=0)
plt.title('Returns Heatmap: Buy vs Sell Thresholds')
plt.xlabel('Sell Threshold')
plt.ylabel('Buy Threshold')
plt.show()

### BUY/SELL OPTIMIZATION

In [9]:
def optimize_percentages(symbol, start_date, end_date, initial_capital=10000):
    # Define the ranges to test
    buy_percentages = np.arange(0.1, 1.0, 0.05)   
    sell_percentages = np.arange(0.1, 1.0, 0.05)  
    
    best_return = -float('inf')
    best_params = None
    results = []
    
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
        if df.empty:
            print(f'No data available for {symbol} in the specified period')
            return None, pd.DataFrame()
    except Exception as e:
        print(f'Error downloading data: {e}')
        return None, pd.DataFrame()
    
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    df = df.dropna()
    
    print(f'Testing {len(buy_percentages) * len(sell_percentages)} combinations...')
    
    for buy_pct in buy_percentages:
        for sell_pct in sell_percentages:
            amount = initial_capital
            position = 0
            shares_held = 0
            trades = []
            
            for i in range(len(df)):
                current_price = float(df['Close'].iloc[i])
                rolling_mean = df['RollingMean_20'].iloc[i]
                rolling_std = df['RollingStd_20'].iloc[i]
                z_score = (current_price - rolling_mean) / rolling_std
                
                if i >= 20:
                    recent_volumes = df['Volume'].iloc[i-20:i]
                    max_vol_20min = float(recent_volumes.max())
                else:
                    max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
                
                current_volume = float(df['Volume'].iloc[i])
                
                signal = buy_sig(z_score, max_vol_20min, current_volume)
                
                if signal == 1 and position == 0:
                    investment = amount * buy_pct
                    shares_to_buy = int(investment // current_price)
                    if shares_to_buy > 0:
                        position = current_price * shares_to_buy
                        shares_held = shares_to_buy
                        amount -= position
                        trades.append({
                            'time': df.index[i],
                            'type': 'BUY',
                            'shares': shares_to_buy,
                            'price': current_price
                        })
                
                elif signal == -1 and position > 0:
                    shares_to_sell = int(shares_held * sell_pct)
                    if shares_to_sell > 0:
                        sale_value = current_price * shares_to_sell
                        amount += sale_value
                        shares_held -= shares_to_sell
                        position = current_price * shares_held
                        trades.append({
                            'time': df.index[i],
                            'type': 'SELL',
                            'shares': shares_to_sell,
                            'price': current_price
                        })
            
            final_position_value = shares_held * float(df['Close'].iloc[-1])
            total_value = amount + final_position_value
            total_return = ((total_value - initial_capital) / initial_capital) * 100
            
            results.append({
                'buy_percentage': buy_pct * 100,  
                'sell_percentage': sell_pct * 100,
                'total_return': total_return,
                'n_trades': len(trades)
            })
            
            if total_return > best_return:
                best_return = total_return
                best_params = {
                    'buy_percentage': buy_pct * 100,
                    'sell_percentage': sell_pct * 100,
                    'total_return': total_return,
                    'n_trades': len(trades)
                }
    
    results_df = pd.DataFrame(results)
    return best_params, results_df

In [10]:
print('\nOptimizing trading percentages...')
print('Note: Using 1-minute data from the last 5 trading days')
best_pct_params, pct_results_df = optimize_percentages(stockin, start_date, end_date, initial_capital=10000)

print('\nBest Percentage Parameters Found:')
print(f"Buy Percentage: {best_pct_params['buy_percentage']:.1f}%")
print(f"Sell Percentage: {best_pct_params['sell_percentage']:.1f}%")
print(f"Total Return: {best_pct_params['total_return']:.2f}%")
print(f"Number of Trades: {best_pct_params['n_trades']}")

print('\nTop 5 Percentage Combinations:')
print(pct_results_df.sort_values('total_return', ascending=False).head())

plt.figure(figsize=(12, 8))
heatmap_data_pct = pct_results_df.pivot(
    index='buy_percentage',
    columns='sell_percentage',
    values='total_return'
)
sns.heatmap(heatmap_data_pct, annot=True, cmap='RdYlGn', center=0, fmt='.1f')
plt.title('Returns Heatmap: Buy vs Sell Percentages')
plt.xlabel('Sell Percentage')
plt.ylabel('Buy Percentage')
plt.show()


Optimizing trading percentages...
Note: Using 1-minute data from the last 5 trading days


NameError: name 'start_date' is not defined

### STOCK, Z-Score, BUY/SELL OPTIMIZATION

## Single Thread

In [12]:
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS']

buy_z_scores = np.arange(-0.3, -0.8, -0.1)
sell_z_scores = np.arange(1.1, 1.4, 0.1)
buy_percentages = np.arange(0.20, 1.00, 0.05)
sell_percentages = np.arange(0.20, 1.00, 0.05)

results = []

global_start_date = datetime.now() - timedelta(days=3)
global_end_date = datetime.now()

for stock in stocks:
    print(f"\nProcessing {stock}...")
    try:
        df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)
        if df.empty:
            print(f"No data for {stock}")
            continue
        df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
        df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
        df = df.dropna()
    except Exception as e:
        print(f"Error downloading {stock}: {e}")
        continue

    for buy_z, sell_z, buy_pct, sell_pct in itertools.product(buy_z_scores, sell_z_scores, buy_percentages, sell_percentages):
        amount = 10000
        position = 0
        shares_held = 0
        for i in range(len(df)):
            current_price = float(df['Close'].iloc[i])
            rolling_mean = df['RollingMean_20'].iloc[i]
            rolling_std = df['RollingStd_20'].iloc[i]
            z_score = (current_price - rolling_mean) / rolling_std
            if i >= 20:
                recent_volumes = df['Volume'].iloc[i-20:i]
                max_vol_20min = float(recent_volumes.max())
            else:
                max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
            current_volume = float(df['Volume'].iloc[i])
            signal = 0
            if z_score <= buy_z:
                signal = 1
            elif z_score >= sell_z:
                signal = -1
            elif (z_score > 0) and (z_score <= sell_z):
                if max_vol_20min > 0 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15:
                    signal = 1
            if signal == 1 and position == 0:
                investment = amount * buy_pct
                shares_to_buy = int(investment // current_price)
                if shares_to_buy > 0:
                    position = current_price * shares_to_buy
                    shares_held = shares_to_buy
                    amount -= position
            elif signal == -1 and position > 0:
                shares_to_sell = int(shares_held * sell_pct)
                if shares_to_sell > 0:
                    sale_value = current_price * shares_to_sell
                    amount += sale_value
                    shares_held -= shares_to_sell
                    position = current_price * shares_held
        final_position_value = shares_held * float(df['Close'].iloc[-1])
        total_value = amount + final_position_value
        total_return = ((total_value - 10000) / 10000) * 100
        results.append({
            'stock': stock,
            'buy_z': buy_z,
            'sell_z': sell_z,
            'buy_pct': buy_pct,
            'sell_pct': sell_pct,
            'total_return': total_return
        })

results_df = pd.DataFrame(results)

top20 = results_df.sort_values('total_return', ascending=False).head(20)
print("\nTop 20 Best Performing Combinations (across all stocks):")
print(top20)

for stock in stocks:
    stock_df = results_df[results_df['stock'] == stock]
    if not stock_df.empty:
        pivot = stock_df.pivot_table(index='buy_z', columns='sell_z', values='total_return', aggfunc='max')
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=False, cmap='RdYlGn', center=0)
        plt.title(f'Returns Heatmap for {stock}\n(Buy Z vs Sell Z, best buy/sell % for each)')
        plt.xlabel('Sell Z-Score')
        plt.ylabel('Buy Z-Score')
        plt.show()

best_stock = top20.iloc[0]['stock']
best_stock_df = results_df[results_df['stock'] == best_stock]
pivot_pct = best_stock_df.pivot_table(index='buy_pct', columns='sell_pct', values='total_return', aggfunc='max')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_pct, annot=False, cmap='RdYlGn', center=0)
plt.title(f'Returns Heatmap for {best_stock}\n(Buy % vs Sell %, best z-scores for each)')
plt.xlabel('Sell Percentage')
plt.ylabel('Buy Percentage')
plt.show()


Processing AAPL...


C:\Users\sauri\AppData\Local\Temp\ipykernel_20896\88596671.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)
C:\Users\sauri\AppData\Local\Temp\ipykernel_20896\88596671.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])
C:\Users\sauri\AppData\Local\Temp\ipykernel_20896\88596671.py:41: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_volume = float(df['Volume'].iloc[i])
C:\Users\sauri\AppData\Local\Temp\ipykernel_20896\88596671.py:40: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  max_vol_20min = float(df['Volume

KeyboardInterrupt: 

## Parallel Threading


In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from concurrent.futures import ThreadPoolExecutor, as_completed


def simulate_strategy(stock, df, buy_z, sell_z, buy_pct, sell_pct):
    amount = 10000
    position = 0
    shares_held = 0
    for i in range(len(df)):
        current_price = float(df['Close'].iloc[i])
        rolling_mean = df['RollingMean_20'].iloc[i]
        rolling_std = df['RollingStd_20'].iloc[i]
        z_score = (current_price - rolling_mean) / rolling_std

        if i >= 20:
            recent_volumes = df['Volume'].iloc[i-20:i]
            max_vol_20min = float(recent_volumes.max())
        else:
            max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0

        current_volume = float(df['Volume'].iloc[i])
        signal = 0
        if z_score <= buy_z:
            signal = 1
        elif z_score >= sell_z:
            signal = -1
        elif (z_score > 0) and (z_score <= sell_z):
            if max_vol_20min > 0 and abs(current_volume - max_vol_20min) / max_vol_20min <= 0.15:
                signal = 1

        if signal == 1 and position == 0:
            investment = amount * buy_pct
            shares_to_buy = int(investment // current_price)
            if shares_to_buy > 0:
                position = current_price * shares_to_buy
                shares_held = shares_to_buy
                amount -= position
        elif signal == -1 and position > 0:
            shares_to_sell = int(shares_held * sell_pct)
            if shares_to_sell > 0:
                sale_value = current_price * shares_to_sell
                amount += sale_value
                shares_held -= shares_to_sell
                position = current_price * shares_held

    final_position_value = shares_held * float(df['Close'].iloc[-1])
    total_value = amount + final_position_value
    total_return = ((total_value - 10000) / 10000) * 100
    if len(results) % 100 == 0:
        pd.DataFrame(results).to_csv("partial_results.csv", index=False)
    return {
        'stock': stock,
        'buy_z': buy_z,
        'sell_z': sell_z,
        'buy_pct': buy_pct,
        'sell_pct': sell_pct,
        'total_return': total_return
    }



# ==== Parameter Setup ====
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS']
buy_z_scores = np.arange(-0.3, -0.8, -0.1)
sell_z_scores = np.arange(1.1, 1.4, 0.1)
buy_percentages = np.arange(0.20, 1.00, 0.05)
sell_percentages = np.arange(0.20, 1.00, 0.05)

global_start_date = datetime.now() - timedelta(days=3)
global_end_date = datetime.now()

# ==== Prepare Tasks ====

idx = 0
total_tasks = len(stocks) * len(buy_z_scores) * len(sell_z_scores) * len(buy_percentages) * len(sell_percentages)

tasks = []
for stock in stocks:
    print(f"\nDownloading {stock}...")
    df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)
    if df.empty:
        print(f"No data for {stock}")
        continue
    df['RollingMean_20'] = df['Close'].shift(1).rolling(window=20).mean()
    df['RollingStd_20'] = df['Close'].shift(1).rolling(window=20).std()
    df = df.dropna()
    for buy_z, sell_z, buy_pct, sell_pct in itertools.product(buy_z_scores, sell_z_scores, buy_percentages, sell_percentages):
        idx += 1
        if idx % 100 == 0:
            print(f"Completed {idx}/{total_tasks}")

        tasks.append((stock, df.copy(), buy_z, sell_z, buy_pct, sell_pct))

# ==== Run Parallel Simulations ====
results = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(simulate_strategy, *task) for task in tasks]
    for future in as_completed(futures):
        try:
            results.append(future.result())
        except Exception as e:
            print(f"Error in task: {e}")

results_df = pd.DataFrame(results)

# ==== Output Top 20 ====
top20 = results_df.sort_values('total_return', ascending=False).head(20)
print("\nTop 20 Best Performing Combinations:")
print(top20)

# ==== Per Stock Heatmaps ====
for stock in stocks:
    stock_df = results_df[results_df['stock'] == stock]
    if not stock_df.empty:
        pivot = stock_df.pivot_table(index='buy_z', columns='sell_z', values='total_return', aggfunc='max')
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=False, cmap='RdYlGn', center=0)
        plt.title(f'Returns Heatmap for {stock}\n(Buy Z vs Sell Z)')
        plt.xlabel('Sell Z-Score')
        plt.ylabel('Buy Z-Score')
        plt.show()

# ==== Heatmap for Best Stock's Buy/Sell Percentages ====
if not top20.empty:
    best_stock = top20.iloc[0]['stock']
    best_stock_df = results_df[results_df['stock'] == best_stock]
    pivot_pct = best_stock_df.pivot_table(index='buy_pct', columns='sell_pct', values='total_return', aggfunc='max')
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_pct, annot=False, cmap='RdYlGn', center=0)
    plt.title(f'Returns Heatmap for {best_stock}\n(Buy % vs Sell %)')
    plt.xlabel('Sell Percentage')
    plt.ylabel('Buy Percentage')
    plt.savefig(f"{stock}_heatmap.png")
    plt.close()



C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)


Completed 100/38400
Completed 200/38400
Completed 300/38400
Completed 400/38400
Completed 500/38400
Completed 600/38400
Completed 700/38400
Completed 800/38400
Completed 900/38400
Completed 1000/38400
Completed 1100/38400
Completed 1200/38400
Completed 1300/38400
Completed 1400/38400
Completed 1500/38400
Completed 1600/38400
Completed 1700/38400
Completed 1800/38400
Completed 1900/38400
Completed 2000/38400
Completed 2100/38400
Completed 2200/38400
Completed 2300/38400
Completed 2400/38400
Completed 2500/38400
Completed 2600/38400
Completed 2700/38400
Completed 2800/38400
Completed 2900/38400
Completed 3000/38400
Completed 3100/38400
Completed 3200/38400
Completed 3300/38400
Completed 3400/38400
Completed 3500/38400
Completed 3600/38400
Completed 3700/38400
Completed 3800/38400



C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)


Completed 3900/38400
Completed 4000/38400
Completed 4100/38400
Completed 4200/38400
Completed 4300/38400
Completed 4400/38400
Completed 4500/38400
Completed 4600/38400
Completed 4700/38400
Completed 4800/38400
Completed 4900/38400
Completed 5000/38400
Completed 5100/38400
Completed 5200/38400
Completed 5300/38400
Completed 5400/38400
Completed 5500/38400
Completed 5600/38400
Completed 5700/38400
Completed 5800/38400
Completed 5900/38400
Completed 6000/38400
Completed 6100/38400
Completed 6200/38400
Completed 6300/38400
Completed 6400/38400
Completed 6500/38400
Completed 6600/38400
Completed 6700/38400
Completed 6800/38400
Completed 6900/38400
Completed 7000/38400
Completed 7100/38400
Completed 7200/38400
Completed 7300/38400
Completed 7400/38400
Completed 7500/38400
Completed 7600/38400



C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)


...

Completed 30800/38400
Completed 30900/38400
Completed 31000/38400
Completed 31100/38400
Completed 31200/38400
Completed 31300/38400
Completed 31400/38400
Completed 31500/38400
Completed 31600/38400
Completed 31700/38400
Completed 31800/38400
Completed 31900/38400
Completed 32000/38400
Completed 32100/38400
Completed 32200/38400
Completed 32300/38400
Completed 32400/38400
Completed 32500/38400
Completed 32600/38400
Completed 32700/38400
Completed 32800/38400
Completed 32900/38400
Completed 33000/38400
Completed 33100/38400
Completed 33200/38400
Completed 33300/38400
Completed 33400/38400
Completed 33500/38400
Completed 33600/38400
Completed 33700/38400
Completed 33800/38400
Completed 33900/38400
Completed 34000/38400
Completed 34100/38400
Completed 34200/38400
Completed 34300/38400
Completed 34400/38400
Completed 34500/38400



C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start=global_start_date, end=global_end_date, interval='1m', progress=False)


Completed 34600/38400
Completed 34700/38400
Completed 34800/38400
Completed 34900/38400
Completed 35000/38400
Completed 35100/38400
Completed 35200/38400
Completed 35300/38400
Completed 35400/38400
Completed 35500/38400
Completed 35600/38400
Completed 35700/38400
Completed 35800/38400
Completed 35900/38400
Completed 36000/38400
Completed 36100/38400
Completed 36200/38400
Completed 36300/38400
Completed 36400/38400
Completed 36500/38400
Completed 36600/38400
Completed 36700/38400
Completed 36800/38400
Completed 36900/38400
Completed 37000/38400
Completed 37100/38400
Completed 37200/38400
Completed 37300/38400
Completed 37400/38400
Completed 37500/38400
Completed 37600/38400
Completed 37700/38400
Completed 37800/38400
Completed 37900/38400
Completed 38000/38400
Completed 38100/38400
Completed 38200/38400
Completed 38300/38400
Completed 38400/38400


C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])
C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_volume = float(df['Volume'].iloc[i])
C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  max_vol_20min = float(df['Volume'].iloc[:i].max()) if i > 0 else 0.0
C:\Users\sauri\AppData\Local\Temp\ipykernel_2124\2340220629.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) inste

Error in task: Length of values (9500) does not match length of index (9501)
Error in task: Length of values (13700) does not match length of index (13703)
Error in task: Length of values (14900) does not match length of index (14901)
Error in task: Length of values (15100) does not match length of index (15103)
Error in task: Length of values (15900) does not match length of index (15901)
Error in task: Length of values (15900) does not match length of index (15901)
Error in task: Length of values (16300) does not match length of index (16301)
Error in task: Length of values (16800) does not match length of index (16802)
Error in task: Length of values (17600) does not match length of index (17601)
Error in task: Length of values (20200) does not match length of index (20201)
Error in task: Length of values (20500) does not match length of index (20501)
Error in task: Length of values (22700) does not match length of index (22701)
